In [4]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori, association_rules

In [2]:
datapath = r'C:\Users\86139\Desktop\西北大学\IEMS308\Association Rules\origin\Dillards POS\trnsact.csv'
reader = pd.read_csv(datapath,header = None,sep=',',error_bad_lines=False, iterator=True,usecols = range(12))



,0,1,2,3,4,5,6,7,8,9,10,11
1,517662,8404,40,1600,0,2005-08-21,P,1,23.5,11.75,11.75,306200032
2,517662,8404,40,1600,0,2005-08-21,P,1,23.5,11.75,11.75,306300032
3,517662,8602,720,500,0,2005-02-28,P,1,23.5,23.50,23.50,223200045
4,517662,8602,720,1600,0,2005-03-03,R,1,23.5,23.50,23.50,859000083
5,517662,8602,720,6000,0,2005-03-26,P,1,23.5,17.62,17.62,658800088
...,...,...,...,...,...,...,...,...,...,...,...,...
96,517714,107,630,900,0,2004-09-22,P,1,29.5,22.13,22.13,372100080
97,517714,107,700,2800,0,2005-03-09,P,1,29.5,29.50,29.50,739400059
98,517714,202,250,400,0,2004-10-06,P,1,29.5,22.12,22.12,779500052
99,517714,202,290,2000,716708892,2005-02-12,P,1,29.5,29.50,29.50,118300162


In [3]:
# read the data in chunk and then merge
store_ID = 9806

iteration_count = 0
training_data = reader.get_chunk(1)
training_data.columns = ['SKU', 'Store', 'Register', 'TRANNUM','SEQ','Saledate','Stype', 'Quantity','MIC','OrigiPrice','AMT','InterID']
while True:
    try:
        iteration_count += 1
        df = reader.get_chunk(10000000)
        print("%d-iteration has been done"%iteration_count)
        df.columns = ['SKU', 'Store', 'TRANNUM', 'Register','SEQ','Saledate','Stype', 'Quantity','MIC','OrigiPrice','AMT','InterID']
        df = df[df['Store'] == store_ID]
        df['SKU'] = df['SKU'].astype('str') 
        df.dropna(axis = 0, subset = ['SKU'], inplace = True)
        df['Store'] = df['Store'].astype('str')
        df = df[~df['Stype'].str.contains('R')]
        training_data = pd.concat([training_data, df])
    except StopIteration:
        print ("Iteration is stopped.")
        break

1-iteration has been done
2-iteration has been done
3-iteration has been done
4-iteration has been done
5-iteration has been done
6-iteration has been done
7-iteration has been done
8-iteration has been done
9-iteration has been done
10-iteration has been done
11-iteration has been done
12-iteration has been done
13-iteration has been done
Iteration is stopped.


In [4]:
training_data = training_data.drop(axis = 0, index = 0)
training_data

,SKU,Store,Register,TRANNUM,SEQ,Saledate,Stype,Quantity,MIC,OrigiPrice,AMT,InterID
1612,387,9806,243,68100,0,2005-02-11,P,1,79.0,79.0,79.0,749500006
1613,387,9806,333,79600,0,2005-07-14,P,1,79.0,79.0,79.0,565700006
1614,387,9806,613,53700,936500134,2005-06-07,P,1,79.0,79.0,79.0,717100004
1615,387,9806,823,77700,191100910,2005-01-14,P,1,79.0,79.0,79.0,360700007
6956,1634,9806,43,90700,395009195,2005-08-18,P,1,60.0,60.0,60.0,920600013
...,...,...,...,...,...,...,...,...,...,...,...,...
120916721,9999950,9806,543,46000,543308797,2005-04-21,P,1,13.5,13.5,13.5,529200067
120916722,9999950,9806,743,73300,405408829,2005-08-04,P,1,14.0,14.0,14.0,659900087
120916723,9999950,9806,843,90700,68700873,2004-12-06,P,1,13.5,13.5,13.5,961200100
120916724,9999950,9806,903,46100,545403473,2005-08-11,P,1,14.0,14.0,14.0,206400090


In [8]:
# groupby training_data
target_data = (training_data.groupby(['Saledate','TRANNUM','SKU'])['Quantity']
                    .sum().unstack().reset_index()
                    .fillna(0))
target_data

SKU,Saledate,Register,1000319,1000331,1000346,1000597,1001027,1001224,1001232,1001712,...,997676,998159,998335,998471,998595,998886,999058,999624,999906,999948
0,2004-08-02,30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2004-08-02,130,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2004-08-02,140,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2004-08-02,180,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2004-08-02,883,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2335,2005-08-26,213,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2336,2005-08-26,340,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2337,2005-08-26,530,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2338,2005-08-27,213,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
# Clean the data
def hot_encode(x): 
    if(x<= 0): 
        return 0
    if(x>= 1): 
        return 1
    
target_encoded = target_data.iloc[: , 3:].applymap(hot_encode) 
target_encoded


SKU,1000331,1000346,1000597,1001027,1001224,1001232,1001712,1001777,1003246,1003366,...,997676,998159,998335,998471,998595,998886,999058,999624,999906,999948
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2335,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2336,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2337,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2338,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
#Apriori
frq_items = apriori(target_encoded, min_support = 0.06, use_colnames = True) 

# Collecting the inferred rules in a dataframe 
rules = association_rules(frq_items, metric ="lift", min_threshold = 1) 
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 
rules.sort_values(by=['lift'], inplace = True, ascending = False)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
29,(878635),(888635),0.082906,0.088034,0.068803,0.829897,9.426984,0.061505,5.361254
28,(888635),(878635),0.088034,0.082906,0.068803,0.781553,9.426984,0.061505,4.198253
35,(828635),"(768635, 748635)",0.116667,0.069231,0.061538,0.527473,7.619048,0.053462,1.969767
30,"(768635, 748635)",(828635),0.069231,0.116667,0.061538,0.888889,7.619048,0.053462,7.950000
33,(768635),"(748635, 828635)",0.113675,0.074786,0.061538,0.541353,7.238668,0.053037,2.017269
32,"(748635, 828635)",(768635),0.074786,0.113675,0.061538,0.822857,7.238668,0.053037,5.003446
31,"(768635, 828635)",(748635),0.090171,0.099145,0.061538,0.682464,6.883478,0.052598,2.837020
34,(748635),"(768635, 828635)",0.099145,0.090171,0.061538,0.620690,6.883478,0.052598,2.398640
26,(768635),(828635),0.113675,0.116667,0.090171,0.793233,6.799141,0.076909,4.272121
27,(828635),(768635),0.116667,0.113675,0.090171,0.772894,6.799141,0.076909,3.902688


In [93]:
# Process profit of each SKU
datapath = r'C:\Users\86139\Desktop\西北大学\IEMS308\Association Rules\origin\Dillards POS\skstinfo.csv'
skst_reader = pd.read_csv(datapath, header = None, sep=',',error_bad_lines=False,iterator=True,
                       usecols = [0,1,2,3])
try:
    skst_data = skst_reader.get_chunk(50000000)
except StopIteration:
    print ("Iteration is stopped.")
    
skst_data.columns = ['SKU', 'StoreID','Cost','Retail']

In [94]:
skst_data = skst_data[skst_data['StoreID'] == store_ID]
skst_data['Profit'] = skst_data['Retail'] - skst_data['Cost']
skst_data['SKU'] = skst_data['SKU'].astype('str')
skst_data

,SKU,StoreID,Cost,Retail,Profit
2205,268,9806,15.75,29.00,13.25
2207,319,9806,11.91,25.50,13.59
2217,327,9806,3.72,3.75,0.03
2612,387,9806,38.00,79.00,41.00
3975,466,9806,8.89,28.00,19.11
...,...,...,...,...,...
39227810,9999863,9806,14.00,42.00,28.00
39228661,9999912,9806,12.36,34.00,21.64
39228734,9999926,9806,62.10,140.00,77.90
39229421,9999950,9806,8.40,14.00,5.60


In [49]:
# add profit in rules
rule_profit = []
for row in rules.iterrows():
    count = 0
    for i in row[1][1]:
        if(skst_data.loc[skst_data['SKU'] == i].empty):
            continue
        else:
            count += skst_data.loc[skst_data['SKU'] == i]['Profit'].values[0]
    rule_profit.append(count)
rules['Profit'] = rule_profit

In [53]:
rules.reset_index().to_csv('C:/Users/86139/Desktop/Rules%d.csv'%store_ID)

In [54]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,Profit
28,(878635),(888635),0.082906,0.088034,0.068803,0.829897,9.426984,0.061505,5.361254,4.18
29,(888635),(878635),0.088034,0.082906,0.068803,0.781553,9.426984,0.061505,4.198253,5.44
33,(828635),"(768635, 748635)",0.116667,0.069231,0.061538,0.527473,7.619048,0.053462,1.969767,11.50
32,"(768635, 748635)",(828635),0.069231,0.116667,0.061538,0.888889,7.619048,0.053462,7.950000,4.18
34,(768635),"(828635, 748635)",0.113675,0.074786,0.061538,0.541353,7.238668,0.053037,2.017269,10.24
31,"(828635, 748635)",(768635),0.074786,0.113675,0.061538,0.822857,7.238668,0.053037,5.003446,5.44
30,"(828635, 768635)",(748635),0.090171,0.099145,0.061538,0.682464,6.883478,0.052598,2.837020,6.06
35,(748635),"(828635, 768635)",0.099145,0.090171,0.061538,0.620690,6.883478,0.052598,2.398640,9.62
27,(768635),(828635),0.113675,0.116667,0.090171,0.793233,6.799141,0.076909,4.272121,4.18
26,(828635),(768635),0.116667,0.113675,0.090171,0.772894,6.799141,0.076909,3.902688,5.44


In [26]:
# Reopen rules for the convience of processing
store_ID = 9806
res_rules = pd.read_csv('C:/Users/86139/Desktop/Rules%d.csv'%store_ID,sep=',')
res_rules.drop(['antecedent support', 'consequent support'],axis = 1, inplace = True)
res_rules

,Unnamed: 0,antecedents,consequents,support,confidence,lift,Profit
0,1,frozenset({'878635'}),frozenset({'888635'}),0.068803,0.829897,9.426984,4.18
1,2,frozenset({'888635'}),frozenset({'878635'}),0.068803,0.781553,9.426984,5.44
2,3,frozenset({'828635'}),"frozenset({'768635', '748635'})",0.061538,0.527473,7.619048,11.50
3,4,"frozenset({'768635', '748635'})",frozenset({'828635'}),0.061538,0.888889,7.619048,4.18
4,5,frozenset({'768635'}),"frozenset({'828635', '748635'})",0.061538,0.541353,7.238668,10.24
5,6,"frozenset({'828635', '748635'})",frozenset({'768635'}),0.061538,0.822857,7.238668,5.44
6,7,"frozenset({'828635', '768635'})",frozenset({'748635'}),0.061538,0.682464,6.883478,6.06
7,8,frozenset({'748635'}),"frozenset({'828635', '768635'})",0.061538,0.620690,6.883478,9.62
8,9,frozenset({'768635'}),frozenset({'828635'}),0.090171,0.793233,6.799141,4.18
9,10,frozenset({'828635'}),frozenset({'768635'}),0.090171,0.772894,6.799141,5.44


In [87]:
# Lost profit
profit_1 = skst_data.loc[skst_data['SKU'] == "243947"]['Profit']
profit_1

Series([], Name: Profit, dtype: float64)

In [48]:
# open SKUinfo table
datapathsku = r'C:\Users\86139\Desktop\西北大学\IEMS308\Association Rules\origin\Dillards POS\skuinfo.csv'
sku_data = pd.read_csv(datapathsku, header = None,sep=',',error_bad_lines=False,
                       usecols = [0,1])
sku_data.columns = ['SKU', 'DeptID']
sku_data['SKU'] = sku_data['SKU'].astype('str')

In [68]:
# get the departments of rules
ant_number = []
con_number = []
for row in res_rules.iterrows():
    con_temp = list(filter(lambda a: a in '0123456789', row[1][2]))
    if(len(con_temp) > 8):
        con = "".join(con_temp[:6])
    else:
        con = "".join(con_temp)
    con_number.append(con)
    ant_temp = list(filter(lambda ch: ch in '0123456789', row[1][1]))
    if(len(ant_temp) > 8):
        ant = "".join(ant_temp[:6])
    else:
        ant = "".join(ant_temp)
    ant_number.append(ant)

ant_dept = []
con_dept = []
for ele in ant_number:
    ant_temp = sku_data.loc[sku_data['SKU'] == ele]['DeptID'].values[0]
    ant_dept.append(ant_temp)
for ele in con_number:
    con_temp = sku_data.loc[sku_data['SKU'] == ele]['DeptID'].values[0]
    con_dept.append(con_temp)

In [73]:
# Clean the rules result
res_rules['ant_dept'] = ant_dept
res_rules['con_dept'] = con_dept
res_rules['Excepted profit'] = res_rules['Profit'] * res_rules['confidence']

In [91]:
res_rules.replace(0, np.nan, inplace = True)
res_rules.drop(['a'], axis = 1, inplace = True)
res_rules

,Unnamed: 0,antecedents,consequents,support,confidence,lift,Profit,ant_dept,con_dept,Excepted profit
0,1,frozenset({'878635'}),frozenset({'888635'}),0.068803,0.829897,9.426984,4.18,6107,6107,3.468969
1,2,frozenset({'888635'}),frozenset({'878635'}),0.068803,0.781553,9.426984,5.44,6107,6107,4.251650
2,3,frozenset({'828635'}),"frozenset({'768635', '748635'})",0.061538,0.527473,7.619048,11.50,6107,6107,6.065934
3,4,"frozenset({'768635', '748635'})",frozenset({'828635'}),0.061538,0.888889,7.619048,4.18,6107,6107,3.715556
4,5,frozenset({'768635'}),"frozenset({'828635', '748635'})",0.061538,0.541353,7.238668,10.24,6107,6107,5.543459
5,6,"frozenset({'828635', '748635'})",frozenset({'768635'}),0.061538,0.822857,7.238668,5.44,6107,6107,4.476343
6,7,"frozenset({'828635', '768635'})",frozenset({'748635'}),0.061538,0.682464,6.883478,6.06,6107,6107,4.135735
7,8,frozenset({'748635'}),"frozenset({'828635', '768635'})",0.061538,0.620690,6.883478,9.62,6107,6107,5.971034
8,9,frozenset({'768635'}),frozenset({'828635'}),0.090171,0.793233,6.799141,4.18,6107,6107,3.315714
9,10,frozenset({'828635'}),frozenset({'768635'}),0.090171,0.772894,6.799141,5.44,6107,6107,4.204542


In [92]:
# Save the final rules
res_rules.to_csv(r'C:\Users\86139\Desktop\西北大学\IEMS308\Association Rules\Report\final_rules.csv', sep=',',na_rep='NULL')

In [7]:
# just for some processing

import pandas as pd

temp = pd.read_csv(r'C:\Users\86139\Desktop\西北大学\IEMS308\Association Rules\Report\final_rules.csv', sep=',')
temp.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis = 1, inplace = True)
temp.describe()

,support,confidence,lift,Profit,ant_dept,con_dept,Excepted profit
count,36.000000,36.000000,36.000000,32.000000,36.000000,36.000000,32.000000
mean,0.066928,0.563193,4.855488,14.158437,3218.111111,3218.111111,6.971499
std,0.007023,0.147129,2.061669,11.030860,2399.735435,2399.735435,3.973814
min,0.061538,0.374026,2.915997,4.180000,1107.000000,1107.000000,3.153017
25%,0.061966,0.436567,3.117895,5.905000,1107.000000,1107.000000,4.187340
50%,0.064530,0.525469,3.858681,10.000000,1107.000000,1107.000000,5.301152
75%,0.069231,0.651385,6.799141,22.500000,6107.000000,6107.000000,9.262987
max,0.090171,0.888889,9.426984,46.300000,6107.000000,6107.000000,17.437662


In [12]:
temp[(temp['Excepted profit'] > 4.18) & (temp['lift'] > 3.11)]

,antecedents,consequents,support,confidence,lift,Profit,ant_dept,con_dept,Excepted profit
1,frozenset({'888635'}),frozenset({'878635'}),0.068803,0.781553,9.426984,5.44,6107,6107,4.251650
2,frozenset({'828635'}),"frozenset({'768635', '748635'})",0.061538,0.527473,7.619048,11.50,6107,6107,6.065934
4,frozenset({'768635'}),"frozenset({'828635', '748635'})",0.061538,0.541353,7.238668,10.24,6107,6107,5.543459
5,"frozenset({'828635', '748635'})",frozenset({'768635'}),0.061538,0.822857,7.238668,5.44,6107,6107,4.476343
7,frozenset({'748635'}),"frozenset({'828635', '768635'})",0.061538,0.620690,6.883478,9.62,6107,6107,5.971034
9,frozenset({'828635'}),frozenset({'768635'}),0.090171,0.772894,6.799141,5.44,6107,6107,4.204542
15,frozenset({'277865'}),frozenset({'1117865'}),0.067094,0.583643,4.434172,22.50,1107,1107,13.131970
16,frozenset({'617865'}),frozenset({'1117865'}),0.066239,0.582707,4.427058,22.50,1107,1107,13.110902
17,frozenset({'1117865'}),frozenset({'617865'}),0.066239,0.503247,4.427058,25.00,1107,1107,12.581169
18,frozenset({'617865'}),frozenset({'570033'}),0.061538,0.541353,3.290304,10.00,1107,1107,5.413534
